# Creating Data Structures

In [1]:
import numpy as np
import pandas as pd
import xarray as xr

xr.set_options(display_expand_data=False)

rng = np.random.default_rng(seed=0)  # we'll use this later

In the last lecture, we looked at the following example Dataset. In most cases Xarray Datasets are created by reading a file. We'll address this in the next lecture. Here we'll learn how to create Xarray objects from scratch

In [2]:
ds = xr.tutorial.load_dataset("air_temperature")
ds

<xarray.Dataset> Size: 31MB
Dimensions:  (lat: 25, time: 2920, lon: 53)
Coordinates:
  * lat      (lat) float32 100B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
  * lon      (lon) float32 212B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * time     (time) datetime64[ns] 23kB 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float64 31MB 241.2 242.5 243.5 ... 296.2 295.7
Attributes:
    Conventions:  COARDS
    title:        4x daily NMC reanalysis (1948)
    description:  Data is from NMC initialized reanalysis\n(4x/day).  These a...
    platform:     Model
    references:   http://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanaly...

## DataArray

The `DataArray` class is used to attach a name, dimension names, labels, and
attributes to an array.

Our goal will be to recreate the `ds.air` DataArray starting with the underlying numpy data

In [3]:
ds.air

<xarray.DataArray 'air' (time: 2920, lat: 25, lon: 53)> Size: 31MB
241.2 242.5 243.5 244.0 244.1 243.9 ... 297.9 297.4 297.2 296.5 296.2 295.7
Coordinates:
  * lat      (lat) float32 100B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
  * lon      (lon) float32 212B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * time     (time) datetime64[ns] 23kB 2013-01-01 ... 2014-12-31T18:00:00
Attributes:
    long_name:     4xDaily Air temperature at sigma level 995
    units:         degK
    precision:     2
    GRIB_id:       11
    GRIB_name:     TMP
    var_desc:      Air temperature
    dataset:       NMC Reanalysis
    level_desc:    Surface
    statistic:     Individual Obs
    parent_stat:   Other
    actual_range:  [185.16 322.1 ]

In [5]:
array = ds.air.data
array

array([[[241.2 , 242.5 , 243.5 , ..., 232.8 , 235.5 , 238.6 ],
        [243.8 , 244.5 , 244.7 , ..., 232.8 , 235.3 , 239.3 ],
        [250.  , 249.8 , 248.89, ..., 233.2 , 236.39, 241.7 ],
        ...,
        [296.6 , 296.2 , 296.4 , ..., 295.4 , 295.1 , 294.7 ],
        [295.9 , 296.2 , 296.79, ..., 295.9 , 295.9 , 295.2 ],
        [296.29, 296.79, 297.1 , ..., 296.9 , 296.79, 296.6 ]],

       [[242.1 , 242.7 , 243.1 , ..., 232.  , 233.6 , 235.8 ],
        [243.6 , 244.1 , 244.2 , ..., 231.  , 232.5 , 235.7 ],
        [253.2 , 252.89, 252.1 , ..., 230.8 , 233.39, 238.5 ],
        ...,
        [296.4 , 295.9 , 296.2 , ..., 295.4 , 295.1 , 294.79],
        [296.2 , 296.7 , 296.79, ..., 295.6 , 295.5 , 295.1 ],
        [296.29, 297.2 , 297.4 , ..., 296.4 , 296.4 , 296.6 ]],

       [[242.3 , 242.2 , 242.3 , ..., 234.3 , 236.1 , 238.7 ],
        [244.6 , 244.39, 244.  , ..., 230.3 , 232.  , 235.7 ],
        [256.2 , 255.5 , 254.2 , ..., 231.2 , 233.2 , 238.2 ],
        ...,
        [295

We do this using the [DataArray](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.html) _constructor_.

In [7]:
xr.DataArray(array)

<xarray.DataArray (dim_0: 2920, dim_1: 25, dim_2: 53)> Size: 31MB
241.2 242.5 243.5 244.0 244.1 243.9 ... 297.9 297.4 297.2 296.5 296.2 295.7
Dimensions without coordinates: dim_0, dim_1, dim_2

This works. Notice that the default dimension names are not so useful: `dim_0`, `dim_1`, `dim_2`


### Dimension Names

We can change this by specifying dimension names in the appropriate order using the `dims` kwarg 

In [8]:
xr.DataArray(array, dims=("time", "lat", "lon"))

<xarray.DataArray (time: 2920, lat: 25, lon: 53)> Size: 31MB
241.2 242.5 243.5 244.0 244.1 243.9 ... 297.9 297.4 297.2 296.5 296.2 295.7
Dimensions without coordinates: time, lat, lon

Much better! But notice we have no entries under "Coordinates".

### Coordinates

While associating names with dimensions (or axes) of an array is quite useful, attaching coordinate labels to DataArrays makes a lot of analysis quite convenient.

First we'll simply add values for `lon` using the `coords` kwarg. For this datasets, longitudes are regularly spaced at 2.5° intervals between 200°E and 330°E.

`coords` takes a dictionary that maps the name of a dimension to one of
- another `DataArray` object
- a tuple of the form `(dims, data, attrs)` where `attrs` is optional. This is
  roughly equivalent to creating a new `DataArray` object with
  `DataArray(dims=dims, data=data, attrs=attrs)`
- a `numpy` array (or anything that can be coerced to one using `numpy.array`).

We'll start with the last one

In [9]:
lon_values = np.arange(200, 331, 2.5)
xr.DataArray(array, dims=("time", "lat", "lon"), coords={"lon": lon_values})

<xarray.DataArray (time: 2920, lat: 25, lon: 53)> Size: 31MB
241.2 242.5 243.5 244.0 244.1 243.9 ... 297.9 297.4 297.2 296.5 296.2 295.7
Coordinates:
  * lon      (lon) float64 424B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
Dimensions without coordinates: time, lat

In [11]:
print(lon_values.shape, array.shape)

(53,) (2920, 25, 53)


Assigning a plain numpy array is equivalent to creating a DataArray with those values and the same dimension  name

In [12]:
lon_da = xr.DataArray(lon_values, dims="lon")
da = xr.DataArray(array, dims=("time", "lat", "lon"), coords={"lon": lon_da})
da

<xarray.DataArray (time: 2920, lat: 25, lon: 53)> Size: 31MB
241.2 242.5 243.5 244.0 244.1 243.9 ... 297.9 297.4 297.2 296.5 296.2 295.7
Coordinates:
  * lon      (lon) float64 424B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
Dimensions without coordinates: time, lat

In [13]:
lon_da

<xarray.DataArray (lon: 53)> Size: 424B
200.0 202.5 205.0 207.5 210.0 212.5 ... 317.5 320.0 322.5 325.0 327.5 330.0
Dimensions without coordinates: lon

We can also assign coordinates after a DataArray has been created.

In [14]:
da.coords["lat"] = np.arange(75, 14.9, -2.5)
da

<xarray.DataArray (time: 2920, lat: 25, lon: 53)> Size: 31MB
241.2 242.5 243.5 244.0 244.1 243.9 ... 297.9 297.4 297.2 296.5 296.2 295.7
Coordinates:
  * lon      (lon) float64 424B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * lat      (lat) float64 200B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
Dimensions without coordinates: time

### Attributes 

Arbitrary attributes can be assigned using the `.attrs` property

In [15]:
da.attrs["attribute"] = "hello"
da

<xarray.DataArray (time: 2920, lat: 25, lon: 53)> Size: 31MB
241.2 242.5 243.5 244.0 244.1 243.9 ... 297.9 297.4 297.2 296.5 296.2 295.7
Coordinates:
  * lon      (lon) float64 424B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * lat      (lat) float64 200B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
Dimensions without coordinates: time
Attributes:
    attribute:  hello

or specified in the constructor

In [16]:
da2 = xr.DataArray(
    array, dims=("time", "lat", "lon"), coords={"lon": lon_da}, attrs={"attribute": "hello"}
)
da2

<xarray.DataArray (time: 2920, lat: 25, lon: 53)> Size: 31MB
241.2 242.5 243.5 244.0 244.1 243.9 ... 297.9 297.4 297.2 296.5 296.2 295.7
Coordinates:
  * lon      (lon) float64 424B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
Dimensions without coordinates: time, lat
Attributes:
    attribute:  hello

### Non-dimension coordinates

Sometimes we want to attach coordinate variables along an existing dimension. Notice that 
1. `itime` is not bolded and 
2. has a name "time" that is different from the dimension name "time"

`itime` is an example of a non-dimension coordinate variable i.e. it is a coordinate variable that does not match a dimension name. Here we demonstrate the "tuple" form of assigninment:  `(dims, data, attrs)`

In [17]:
da.coords["itime"] = ("time", np.arange(2920), {"name": "value"})
da

<xarray.DataArray (time: 2920, lat: 25, lon: 53)> Size: 31MB
241.2 242.5 243.5 244.0 244.1 243.9 ... 297.9 297.4 297.2 296.5 296.2 295.7
Coordinates:
  * lon      (lon) float64 424B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * lat      (lat) float64 200B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
    itime    (time) int64 23kB 0 1 2 3 4 5 6 ... 2914 2915 2916 2917 2918 2919
Dimensions without coordinates: time
Attributes:
    attribute:  hello

### Exercises

create a `DataArray` named "height" from random data `rng.random((180, 360)) * 400`

1. with dimensions named "latitude" and "longitude"


In [19]:
height = rng.random((180, 360)) * 400
print(height.shape)
xr.DataArray(height, dims=("latitude", "longitude"), name="height")

(180, 360)


<xarray.DataArray 'height' (latitude: 180, longitude: 360)> Size: 518kB
192.4 101.9 45.38 76.42 56.98 3.814 ... 16.73 47.68 199.2 47.88 303.5 121.3
Dimensions without coordinates: latitude, longitude

2. with dimension coordinates:

- "latitude": -90 to 89 with step size 1
- "longitude": -180 to 179 with step size 1


In [25]:
lat = np.arange(-90, 90, 1)
lon = np.arange(-180, 180, 1)
print(height.size, height.shape, lat.shape, lon.shape)
xr.DataArray(
    height,
    dims=("latitude", "longitude"),
    coords={"latitude": lat, "longitude": lon},
)

64800 (180, 360) (180,) (360,)


<xarray.DataArray (latitude: 180, longitude: 360)> Size: 518kB
192.4 101.9 45.38 76.42 56.98 3.814 ... 16.73 47.68 199.2 47.88 303.5 121.3
Coordinates:
  * latitude   (latitude) int64 1kB -90 -89 -88 -87 -86 -85 ... 85 86 87 88 89
  * longitude  (longitude) int64 3kB -180 -179 -178 -177 ... 176 177 178 179

3. with metadata for both data and coordinates:

- height: "type": "ellipsoid"
- latitude: "type": "geodetic"
- longitude: "prime_meridian": "greenwich"


In [26]:
xr.DataArray(
    height,
    dims=("latitude", "longitude"),
    coords={
        "latitude": ("latitude", lat, {"type": "geodetic"}),
        "longitude": (
            "longitude",
            lon,
            {"prime_meridian": "greenwich"},
        ),
    },
    attrs={"type": "ellipsoid"},
    name="height",
)

<xarray.DataArray 'height' (latitude: 180, longitude: 360)> Size: 518kB
192.4 101.9 45.38 76.42 56.98 3.814 ... 16.73 47.68 199.2 47.88 303.5 121.3
Coordinates:
  * latitude   (latitude) int64 1kB -90 -89 -88 -87 -86 -85 ... 85 86 87 88 89
  * longitude  (longitude) int64 3kB -180 -179 -178 -177 ... 176 177 178 179
Attributes:
    type:     ellipsoid

## Dataset

`Dataset` objects collect multiple data variables, each with possibly different
dimensions.

The constructor of `Dataset` takes three parameters:

- `data_vars`: dict-like mapping names to values. Values are either `DataArray` objects
  or defined with tuples consisting of of dimension names and arrays.
- `coords`: same as for `DataArray`
- `attrs`: same as for `Dataset`

Creating an empty Dataset is easy!

In [27]:
xr.Dataset()

<xarray.Dataset> Size: 0B
Dimensions:  ()
Data variables:
    *empty*

### Data Variables

Let's create a `Dataset` with two data variables: `da` and `da2`

In [28]:
ds = xr.Dataset({"air": da, "air2": da2})
ds

<xarray.Dataset> Size: 62MB
Dimensions:  (lon: 53, lat: 25, time: 2920)
Coordinates:
  * lon      (lon) float64 424B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * lat      (lat) float64 200B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
    itime    (time) int64 23kB 0 1 2 3 4 5 6 ... 2914 2915 2916 2917 2918 2919
Dimensions without coordinates: time
Data variables:
    air      (time, lat, lon) float64 31MB 241.2 242.5 243.5 ... 296.2 295.7
    air2     (time, lat, lon) float64 31MB 241.2 242.5 243.5 ... 296.2 295.7

You can directly assign a new data variables

In [29]:
ds["air3"] = da
ds

<xarray.Dataset> Size: 93MB
Dimensions:  (lon: 53, lat: 25, time: 2920)
Coordinates:
  * lon      (lon) float64 424B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * lat      (lat) float64 200B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
    itime    (time) int64 23kB 0 1 2 3 4 5 6 ... 2914 2915 2916 2917 2918 2919
Dimensions without coordinates: time
Data variables:
    air      (time, lat, lon) float64 31MB 241.2 242.5 243.5 ... 296.2 295.7
    air2     (time, lat, lon) float64 31MB 241.2 242.5 243.5 ... 296.2 295.7
    air3     (time, lat, lon) float64 31MB 241.2 242.5 243.5 ... 296.2 295.7

### Coordinates

Coordinate variables can be assigned using the `coords` kwarg to `xr.Dataset`. Here we use `date_range` from pandas to create a time vector

In [31]:
xr.Dataset(
    {"air": da, "air2": da2},
    coords={"time": pd.date_range("2013-01-01", "2014-12-31 18:00", freq="6h")},
)

<xarray.Dataset> Size: 62MB
Dimensions:  (lon: 53, lat: 25, time: 2920)
Coordinates:
  * lon      (lon) float64 424B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * lat      (lat) float64 200B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
    itime    (time) int64 23kB 0 1 2 3 4 5 6 ... 2914 2915 2916 2917 2918 2919
  * time     (time) datetime64[ns] 23kB 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float64 31MB 241.2 242.5 243.5 ... 296.2 295.7
    air2     (time, lat, lon) float64 31MB 241.2 242.5 243.5 ... 296.2 295.7

Again we can assign coordinate variables after a Dataset has been created.

In [33]:
ds

<xarray.Dataset> Size: 93MB
Dimensions:  (lon: 53, lat: 25, time: 2920)
Coordinates:
  * lon      (lon) float64 424B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * lat      (lat) float64 200B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
    itime    (time) int64 23kB 0 1 2 3 4 5 6 ... 2914 2915 2916 2917 2918 2919
  * time     (time) datetime64[ns] 23kB 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float64 31MB 241.2 242.5 243.5 ... 296.2 295.7
    air2     (time, lat, lon) float64 31MB 241.2 242.5 243.5 ... 296.2 295.7
    air3     (time, lat, lon) float64 31MB 241.2 242.5 243.5 ... 296.2 295.7

In [32]:
ds.coords["time"] = pd.date_range("2013-01-01", "2014-12-31 18:00", freq="6h")
ds

<xarray.Dataset> Size: 93MB
Dimensions:  (lon: 53, lat: 25, time: 2920)
Coordinates:
  * lon      (lon) float64 424B 200.0 202.5 205.0 207.5 ... 325.0 327.5 330.0
  * lat      (lat) float64 200B 75.0 72.5 70.0 67.5 65.0 ... 22.5 20.0 17.5 15.0
    itime    (time) int64 23kB 0 1 2 3 4 5 6 ... 2914 2915 2916 2917 2918 2919
  * time     (time) datetime64[ns] 23kB 2013-01-01 ... 2014-12-31T18:00:00
Data variables:
    air      (time, lat, lon) float64 31MB 241.2 242.5 243.5 ... 296.2 295.7
    air2     (time, lat, lon) float64 31MB 241.2 242.5 243.5 ... 296.2 295.7
    air3     (time, lat, lon) float64 31MB 241.2 242.5 243.5 ... 296.2 295.7

### Attributes

In [ ]:
xr.Dataset(
    {"air": da, "air2": da2},
    coords={"time": pd.date_range("2013-01-01", "2014-12-31 18:00", freq="6h")},
    attrs={"key0": "value0"},
)

In [ ]:
ds.attrs["key"] = "value"

### Exercises

1. create a Dataset with two variables along `latitude` and `longitude`:
   `height` and `gravity_anomaly`


In [35]:
height = rng.random((180, 360)) * 400
gravity_anomaly = rng.random((180, 360)) * 400 - 200

In [36]:
xr.Dataset(
    {
        "height": (("latitude", "longitude"), height),
        "gravity_anomaly": (("latitude", "longitude"), gravity_anomaly),
    }
)

<xarray.Dataset> Size: 1MB
Dimensions:          (latitude: 180, longitude: 360)
Dimensions without coordinates: latitude, longitude
Data variables:
    height           (latitude, longitude) float64 518kB 376.5 224.4 ... 46.98
    gravity_anomaly  (latitude, longitude) float64 518kB -159.9 ... -198.9

2. add coordinates to `latitude` and `longitude`:

- `latitude`: from -90 to 90 with step size 1
- `longitude`: from -180 to 180 with step size 1


In [ ]:
xr.Dataset(
    {
        "height": (("latitude", "longitude"), height),
        "gravity_anomaly": (("latitude", "longitude"), gravity_anomaly),
    },
    coords={
        "latitude": ("latitude", np.arange(-90, 90, 1)),
        "longitude": ("longitude", np.arange(-180, 180, 1)),
    },
)

3. add metadata to coordinates and variables:

- `latitude`: "type": "geodetic"
- `longitude`: "prime_meridian": "greenwich"
- `height`: "ellipsoid": "wgs84"
- `gravity_anomaly`: "ellipsoid": "grs80"


In [37]:
xr.Dataset(
    {
        "height": (("latitude", "longitude"), height, {"ellipsoid": "wgs84"}),
        "gravity_anomaly": (("latitude", "longitude"), gravity_anomaly, {"ellipsoid": "grs80"}),
    },
    coords={
        "latitude": ("latitude", np.arange(-90, 90, 1), {"type": "geodetic"}),
        "longitude": (
            "longitude",
            np.arange(-180, 180, 1),
            {"prime_meridian": "greenwich"},
        ),
    },
)

<xarray.Dataset> Size: 1MB
Dimensions:          (latitude: 180, longitude: 360)
Coordinates:
  * latitude         (latitude) int64 1kB -90 -89 -88 -87 -86 ... 85 86 87 88 89
  * longitude        (longitude) int64 3kB -180 -179 -178 -177 ... 177 178 179
Data variables:
    height           (latitude, longitude) float64 518kB 376.5 224.4 ... 46.98
    gravity_anomaly  (latitude, longitude) float64 518kB -159.9 ... -198.9